# Conditionals

## Input

We will now introduce a special instruction, which adds a fully new dimension to our programming language. This instruction will make our programs interactive and capable of handling a dialogue with the user. This will make our programs far more interesting, but at the same time more complex: users are unpredictable, and we can never determine beforehand what the user input will look like. Programs handling input should take this substantial unpredictability into account in the form of additional validation statements, which translates into added complexity of our programs.

We will define a single instruction, `input`, which "somehow receives" a string from the user. We will not focus on the actual mechanism: whether it is an input dialog box, a line typed in a terminal, or something else, it does not matter to us. What does matter is that we expect to find the data from the next user input somewhere in the state.

We will therefore assume that, from now on, the state "contains" the sequence of all inputs that the user will ever produce for the product. This makes our state an "oracle" of sorts, because it knows in advance what will happen with the user. This "trick" is needed when defining how we manage input on paper, because of the fundamental limitation of paper not being an interactive system. This assumption is also not completely ungrounded: we assume that we already know the inputs, but these inputs will indeed be known at some point in time. So what we really do is define how the program did behave, or would behave, given inputs that **did** happen or **would** happen. These past/speculiative inputs will be stored in the state. The new structure of the state will therefore be:

$$\{ \dots, \text{input} := \{ \text{next} := \text{"}\dots\text{"}, \text{rest} := \{ \text{next} := \text{"}\dots\text{"}, \text{rest} := \{\dots \} \} \} \} $$

Where the first $\dots$ are just the regular data we would keep in the state, and inside `input` we find a (potentially infinite) stream of values, each a string. A possible state with input will therefore look like:

$$\{ \text{input} := \{ \text{next} := \text{"hello"}, \text{rest} := \{ \text{next} := \text{"world"} \}$$

We can therefore just read the next user input by simply stating `S[input][next]`, and we can discard it and move to the next user input by `S[input := S[input][rest]]`.

For example, with respect to the state example above, we could read the current input by stating:

$$\{ \text{input} := \{ \text{next} := \text{"hello"}, \text{rest} := \{ \text{next} := \text{"world"} \}[\text{input}][\text{next}] \rightarrow \text{"hello"}$$

The susequent input could be read as:

$$\{ \text{input} := \{ \text{next} := \text{"hello"}, \text{rest} := \{ \text{next} := \text{"world"} \}[\text{input}][\text{rest}][\text{next}] \rightarrow \text{"world"}$$

The semantics of `input` are just:

`eval( v := input(), S)` $\rightarrow$ `<done>, S'`

where `S[v := S[input][next]][input := S[input][rest]]` $$ `S'`


[[[EXAMPLE PROGRAMS
- `x := input(); y := input(); z := x + " " + y + "!"; done` where $input := \{ next := \text{"Hello"}, rest := \{ next := \text{"World"}, rest := \{ \dots \} \} \}$
    - `x := input(); y := input(); z := int(x) + int(y); done` where $input := \{ next := \text{"3"}, rest := \{ next := \text{"5"}, rest := \{ \dots \} \} \}$
]]]

## Conditionals
An obvious and immediate consequence of input is that it makes our programs less predictable. We have no access to the input as we are writing the program, and so the program must be able to adjust its behaviour consistently with respect to the inputs it will receive.

Some instructions will, at some point during the evaluation of a program, become a better response than others. This introduces a requirement: our program must be able to choose a statement rather than another based on some dynamically (i.e. only known at runtime) condition. To achieve this goal we can introduce the fundamental instruction to perform choices, known as the *conditional* instruction or *if-then-else*.

The syntax if this instruction is `if C then P else Q`, where `C` is an expression, and both `P` and `Q` are statements. We expect `C` to be a boolean expression, that is an expression which will evaluate to either `True` or `False`. Should `C` evaluate to something else, we will interrupt evaluation as the program gets stuck in an error state. 

When `C` evaluates to `True`, we will then move onto the evaluation of `P`, thereby skipping `Q`. Conversely, when `C` evaluates to `False`, we will then move onto the evaluation of `Q`, thereby skipping `P`. The semantics of `if-then-else` reflect this process and are not particularly intricated:
- `eval(<if True then p else q>, S)` $\rightarrow$ `(<p>, S)`
- `eval(<if False then p else q>, S)` $\rightarrow$ `(<q>, S)`
- `eval(<if C then p else q>, S)` $\not\rightarrow$ where `C` is any non-boolean constant value)
- `eval(<if c then p else q>, S)` $\rightarrow$ `(<if c' then p else q>, S)` where `eval_expr(<c>, S)` $\rightarrow$  `<c'>`


- let us see what happens when we use a conditional in a program
    - we will use curly brackets `{` and `}` to denote the boundaries of `p` and `q`
    - `x := int(input()); if x > 0 then { z := "the input was positive"; done } else { z := "the input was not positive"; done }; done`
    - `x := int(input()); if x > 0 then { z := "the input was positive"; done } else { if x < 0 then { z := "the input was negative"; done } else { z := the input was zero; done} }; done`
- let us improve readability of our programs: this only affects the syntax of the program, but not the semantics
    - `x := int(input()); if x > 0 then { z := "the input was positive"; done } else { if x < 0 then { z := "the input was negative"; done } else { z := the input was zero; done} }; done` is getting hard to read
    - the `done` instruction will remain implicit
        - `x := int(input()); if x > 0 then { z := "the input was positive" } else { if x < 0 then { z := "the input was negative" } else { z := the input was zero } }` is already a bit shorter
    - instead of `;` we will use new lines (and also for the `then` and `else` sub-programs):

```
x := int(input())
if x > 0 then { 
z := "the input was positive" 
} else { 
if x < 0 then { 
z := "the input was negative" 
} else { 
z := the input was zero } }
```
- this is getting almost readable...
    - instead of `{` and `}` we will use indentation, that is move sub-programs to the right (with spaces/tabs) in order to specify nesting
    - we get to the much prettier
        
```
x := int(input())
if x > 0 then
  z := "the input was positive" 
else
  if x < 0 then 
    z := "the input was negative" 
  else
    z := the input was zero
```
- depending on what we are doing, we will use one or the other syntax (one-liner vs formatted) without thinking about it
    - repeat after me: *changes in syntax are not relevant, it is the semantics that count*

- sometimes we need to chain multiple decisions together:
    - note: the operator `==` between two strings returns `True` if the two strings are exactly equal, `False` if they are not equal
```
todo := input()
x := int(input())
y := int(input())
if todo == "add" then
    z := x + y
else
    if todo == "sub" then
        z := x - y
    else
        if todo == "mul" then
            z := x * y
        else
            if todo == "div" then
                z := x / y
            else
                z := 0
```
- fortunately, many languages nowadays offer a simple way to put an `if` as the only instruction inside an `else`: `elif`/`elseif`:
```
todo := input()
x := int(input())
y := int(input())
if todo == "add" then
  z := x + y
elif todo == "sub" then
  z := x - y
elif todo == "mul" then
  z := x * y
elif todo == "div" then
  z := x / y
else
  z := 0
```
- what is the expressive power of conditional instructions?
    - let us draw again the graph of all possible program traces
        - `p` and `q` are possible programs, so they have traces as well
        - `if c then p else q` chooses which trace to jump to
    - the traces of a conditional instruction are all the traces of the then-sub-programs and all the traces of the else-sub-program, thus the total number of traces is the *sum* of the number of traces of the sub-programs
    - show a simple example and its two possible traces: `x := int(input()); if x = 0 then z := "yes" else z := "no"`
- what happens to the number of traces when we combine two conditionals sequentially?
    - show a simple example and its four possible traces: `x := int(input()); if x % 2 = 0 then { z := "yes" } else { z := "no" }; if x % 3 = 0 then { w := "yes" } else { w := "no" }`
    - the traces of two programs sequenced together are the traces of the first sub-program followed by the traces of the second sub-program, thus the total number of traces is the *product* of the number of traces of the sub-programs
    - $5$ conditional statements sequenced together have $2^5 = 32$ total possible traces
    - $10$ conditional statements sequenced together have $2^{10} = 1024$ total possible traces!
- what about `elif` nesting?
    - show a simple example and its three possible traces: `x := int(input()); if x % 2 = 0 then { z := "yes" } elif x % 3 = 0 then { z := "almost" } else { z := "no" }`
    - number of traces follows from definition of elif as else branch with a single if statement.
- let us produce an example with five possible traces: how would we go about it?
   - let us count the paths to be sure 

```
if c1 then 
  p1
else
  if c2 then
    p2
  else
    q2
  if c3 then
    p3
  else
    q3
```
  
- in conclusion, `input` gives us unpredictability, which we turn into possibilities with conditional statements: conditionals have the potential to duplicate traces, that is each conditional makes our program actually capable of behaving like two separate programs depending on what happens before it is evaluated


### A note about empty `else`-clauses
- sometimes it might be convenient not to execute any sub-program in the `else` branch. With only the rules introuduced so far we would have to write `if C then p else {done}`. We introduce an `if-then` statement to avoid this cumbersome notation.
    - `eval(<if True then p>, S)` $\rightarrow$ `(<p>, S)`
    - `eval(<if False then p>, S)` $\rightarrow$ `(<done>, S)`
    - `eval(<if C then p>, S)` $\rightarrow$ `error` (`C` is any non-boolean constant value)
    - `eval(<if c then p>, S)` $\rightarrow$ `(<if c' then p>, S)` where `eval_expr(<c>, S)` $\rightarrow$ `<c'>`


## Conditional expressions
- we can also use conditionals inside expressions
- in this case, the same conditional is not seen as a statement, but as an expression such as `3`, `3 + 2`, etc.
- this would lead us to code like: `x := if x > 0 then 2 * x else -2 * x`
- not all languages use the same syntax for statement and expression conditional, but the most advanced (ML, Haskell, F#) do 
- other syntaxes:
    - `C ? T : E`
    - `T if C else E`
    - ...
- the `else` clause in conditional statements is optional: if it is not there, then we automatically add `else done`
- the `else` clause in conditional expressions is not optional, since we need a value for both cases of the condition
- the semantics of conditional expressions are defined by `eval_expr`, not `eval`:
    - `eval_expr(<if True then T else E>, S)` $\rightarrow$ `<T>`
    - `eval_expr(<if False then T else E>, S)` $\rightarrow$ `<E>`
    - `eval_expr(<if C then T else E>, S)` $\rightarrow$ `<if C' then T else E>` (when `C` is an expression and `eval_expr(<C>, S)` $\rightarrow$ `<C'>`)
- examples of evaluation of conditional expressions such as `x := 10; y := if x > 0 then 2 * x + 1 else 0; done`